In [1]:
import os
import glob
import pandas as pd 
import geopandas as gpd
import geoparquet as gpq
from utils import *
from congestion_metrics import *
import logging
# from simpledbf import Dbf5

In [2]:
msm_link_volumes = gpd.read_file('/Users/tszchun.chow/Documents/GitHub/akl_touch/outputs/congestion_metrics/msm_link_volumes.gpkg')# .columns
msm_link_volumes

,ID,INODE,JNODE,LENGTH,TYPE,MODES,LANES,NODE_ID,MSM2018,Sector_3,...,@hbw7_26372,@hbw7p_26372,@hbsh7_26372,@hbsh7p_26372,@hcv7_26372,@toll_26372,@veh_26372,@cars_26372,LANE_KM,geometry
0,1-10434,1,10434,8.947524,1,abw,1.0,10434,1.0,Rodney,...,75.565842,83.878082,4.620163,7.299857,8.675142,0.0,0.0,150.24178,8.947524,"LINESTRING (1744722.512 5988249.515, 1735792 5..."
1,1-10448,1,10448,5.237727,1,abw,1.0,10448,1.0,Rodney,...,65.573097,72.786140,17.691942,27.953270,38.213120,0.0,0.0,193.41933,5.237727,"LINESTRING (1744722.512 5988249.515, 1747206 5..."
2,2-10425,2,10425,0.674503,1,abw,1.0,10425,2.0,Rodney,...,85.931511,95.383980,12.725586,20.106426,19.293354,0.0,0.0,223.06308,0.674503,"LINESTRING (1736211.418 5982441.177, 1736501 5..."
3,2-10427,2,10427,0.857741,1,abw,1.0,10427,3.0,Rodney,...,98.677376,109.531890,20.421360,32.265751,4.149139,0.0,0.0,216.94730,0.857741,"LINESTRING (1736211.418 5982441.177, 1736562 5..."
4,3-10427,3,10427,2.108739,1,abw,1.0,10427,3.0,Rodney,...,33.127838,36.771900,9.985406,15.776942,6.821815,0.0,0.0,131.42288,2.108739,"LINESTRING (1737358.501 5985176.528, 1736562 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19793,15105-13058,15105,13058,0.087201,7,r,1.0,13058,560.0,Franklin,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.087201,"LINESTRING (1772305 5890125, 1772357 5890195)"
19794,15105-15104,15105,15104,0.040224,7,r,1.0,15104,560.0,Franklin,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.040224,"LINESTRING (1772305 5890125, 1772282 5890092)"
19795,5242-7334,5242,7334,0.911615,16,a,1.0,7334,264.0,City Fringe East,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.911615,"LINESTRING (1758186 5920245, 1757655 5919504)"
19796,5340-7334,5340,7334,0.513016,16,a,1.0,7334,264.0,City Fringe East,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.513016,"LINESTRING (1757856 5919976, 1757655 5919504)"


In [25]:
emme_links_dir = '../inputs/emme_links'
counter = 0
for filename in os.listdir(emme_links_dir):
    if (('emme_links_26' in filename) and ('16' in filename)):
        period = filename.replace(".gpkg", "").split("_")[-1][2]
        temp_df = gpd.read_file(os.path.join(emme_links_dir, filename))[['ID', '@v0']].rename(columns = {'@v0': f'ffs_{period}'})
        # temp_df.columns = ['ID', f'v0_{period}']
        if counter == 0: 
            ffs_16_df = temp_df.copy()
        else:
            ffs_16_df = ffs_16_df.merge(temp_df, on='ID')
        counter += 1

counter = 0
for filename in os.listdir(emme_links_dir):
    if (('emme_links_26' in filename) and ('76' in filename)):
        period = filename.replace(".gpkg", "").split("_")[-1][2]
        temp_df = gpd.read_file(os.path.join(emme_links_dir, filename))[['ID', '@v0']].rename(columns = {'@v0': f'ffs_{period}'})
        # temp_df.columns = ['ID', f'v0_{period}']
        if counter == 0: 
            ffs_76_df = temp_df.copy()
        else:
            ffs_76_df = ffs_76_df.merge(temp_df, on='ID')
        counter += 1

ffs_df = pd.concat([ffs_16_df ,ffs_76_df]).drop_duplicates(subset=['ID'], keep='first')
ffs_df

,ID,ffs_2,ffs_3,ffs_1
0,1-10434,40.0,40.0,40.0
1,1-10448,40.0,40.0,40.0
2,2-10425,40.0,40.0,40.0
3,2-10427,40.0,40.0,40.0
4,3-10427,40.0,40.0,40.0
...,...,...,...,...
19793,15105-13058,60.0,60.0,60.0
19794,15105-15104,60.0,60.0,60.0
6317,5242-7334,55.0,55.0,55.0
6541,5340-7334,55.0,55.0,55.0


In [29]:
key_cols = ['ID', 'INODE', 'JNODE','LENGTH', 'TYPE', 'MODES', 'LANES', 'NODE_ID',
            'MSM2018','Sector_3', 'Link_Type','Description', 'Group_1', 'Group_2', 'Display'] 
baseline_cols = ['VOLAX_26116', 'VOLAU_26116', 'TIMAU_26116', 'vcv_26116', 'VOLAX_26316', 'VOLAU_26316', 'TIMAU_26316', 'vcv_26316']

CONGESTED_MAP = {11: 0.82, 12: 0.82, 13: 0.82, 14: 0.82, 15: 0.82, 16: 0.82, 17: 0.82, 
                 18: 0.9, 19: 0.9, 20: 0.9, 21: 0.9,
                 22: 0.46,
                 23: 0.58, 24: 0.58, 25: 0.58, 26: 0.58, 27: 0.58}

0.82

In [38]:
msm_link_baseline = msm_link_volumes[key_cols + baseline_cols + ['geometry']].merge(ffs_df, on='ID')
# msm_link_baseline['CONGESTION_VCV'] = msm_link_baseline['TYPE'].map(CONGESTED_MAP).fillna(99)
msm_link_baseline['cong_26116'] = (msm_link_baseline['vcv_26116'] >= msm_link_baseline['TYPE'].map(CONGESTED_MAP).fillna(99)).astype(int)
msm_link_baseline['cong_26316'] = (msm_link_baseline['vcv_26316'] >= msm_link_baseline['TYPE'].map(CONGESTED_MAP).fillna(99)).astype(int)
msm_link_baseline.loc[msm_link_baseline['vcv_26116']>=1, 'cong_26116'] = 2
msm_link_baseline.loc[msm_link_baseline['vcv_26316']>=1, 'cong_26316'] = 2
msm_link_baseline.to_file('../outputs/congestion_metrics/baseline_link_volumes.shp')


/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_79881/2141596918.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  msm_link_baseline.to_file('../outputs/congestion_metrics/baseline_link_volumes.shp')
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Description' to 'Descriptio'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'VOLAX_26116' to 'VOLAX_2611'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'VOLAU_26116' to 'VOLAU_2611'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-to

In [43]:
simplified_cols = [x for x in msm_link_volumes.columns.tolist() if ((('@h') not in x) and (('@e') not in x) and (('@veh') not in x) and (('VOLAD') not in x) and (('VOLAD') not in x))]

msm_link_volumes[simplified_cols].drop(columns='geometry').to_csv(f'{CONGESTION_METRICS_DIR}/msm_links_volumes_csv.csv', index=None)

In [47]:
toll_cols = ['ID', 'INODE', 'JNODE'] + [x for x in msm_link_volumes.columns.tolist() if 'toll' in x] + ['geometry']
msm_link_volumes[toll_cols].to_file(f'{CONGESTION_METRICS_DIR}/msm_links_tolls.gpkg')

'''

In [9]:
## read nodes and links
def year(scen_code):
    if scen_code == 12:
        return 18
    else:
        return YEAR



def read_nodes_and_links():
    
    msm_links_dfs = {}
    counter = 0 

    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        temp_dfs = []
        for scenario in BASE_SCENARIO_MAP:
            scen_code = BASE_SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_dfs.append(gpd.read_file(file)[ROAD_LINKS_BASE_COLS])
                    elif file.endswith('emme_nodes.shp'):
                        # do once only
                        msm_nodes_df = gpd.read_file(file)
                        counter += 1

        msm_links_dfs[period] = pd.concat(temp_dfs).drop_duplicates(subset=['ID', 'INODE', 'JNODE'])
    
    return msm_links_dfs, msm_nodes_df

msm_links_dfs, msm_nodes_df = read_nodes_and_links()

ValueError: No objects to concatenate

In [4]:
# pnr_zones = msm_nodes_df[msm_nodes_df['ID']>90000]
# pnr_zones.sjoin(MSM_ZONES, predicate='within')[['ID', 'MSM2018', 'LBA_Name', 'Sector_1', 'Sector_2', 'Sector_3']]# .to_csv(os.path.join(INPUT_DIR, 'PnR_Zones.csv'))

In [3]:
def map_nodes_to_zone(msm_nodes_df):
    return msm_nodes_df.sjoin_nearest(MSM_ZONES, how='left')[['ID', 'MSM2018', 'Sector_3']].rename(columns={'ID': 'NODE_ID'})
    # return msm_nodes_df.sjoin(MSM_ZONES, how='left', predicate='within')[['ID', 'MSM2018', 'Sector_3']].rename(columns={'ID': 'NODE_ID'})

def map_links(msm_links_dfs, node_zone_map):
    mapped_links_dfs = {}
    for period in PERIOD_MAP:
        mapped_links_dfs[period] = msm_links_dfs[period].copy().merge(node_zone_map, left_on='JNODE', right_on='NODE_ID')
    return mapped_links_dfs

def filter_road_links(msm_links_dfs):
    msm_road_links_dfs = {}
    for period in PERIOD_MAP:
        temp_df = msm_links_dfs[period][msm_links_dfs[period]['TYPE'] >= 0]
        msm_road_links_dfs[period] = temp_df.merge(LINK_TYPE_MAP, left_on='TYPE', right_on='Link_Type', how='left')

    return msm_road_links_dfs

node_zone_map = map_nodes_to_zone(msm_nodes_df)
msm_road_links_dfs = filter_road_links(map_links(msm_links_dfs, node_zone_map))

In [8]:
LINK_TYPE_MAP

,Link_Type,Description,Group_1,Group_2,Display
0,1,Centroid Connector,Connector,Connector,NaN
1,2,"Walk-only links, including centroid connectors",Walk,Walk,PT
2,3,"Multi-mode PT access, p-connectors",PT Access,PT Access,PT
3,4,Bus-only links,Bus,Bus,PT
4,5,Ferry links,Ferry,Ferry,PT
5,6,Ferry links,Ferry,Ferry,PT
6,7,Rail links,Rail,Rail,PT
7,8,-,NaN,NaN,NaN
8,9,-,NaN,NaN,NaN
9,10,LRT-only links,LRT,LRT,PT


In [4]:
ROAD_LINKS_VOL_COLS =  ['ID', 'VOLAX', 'VOLAU', 'VOLAD', 'TIMAU', '@vcv', '@eb7', '@eb7p', '@hbw7', '@hbw7p', '@hbsh7', '@hbsh7p', '@hcv7', '@toll', '@veh', '@cars']
PT_LINKS_VOL_COLS = ['ID', 'VOLTR', 'TIMTR']

In [5]:
def rename_columns(suffix, original_cols = ROAD_LINKS_VOL_COLS, exceptions = ["ID"]):
    new_cols = {}
    for col in original_cols:
        if col not in exceptions:
            new_cols[col] = col + "_" + suffix
        else: 
            new_cols[col] = col 
    return new_cols

def get_road_volume_time_by_periods(links_dfs):
    link_vol_dfs = {}
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        base_df = links_dfs[period].copy()
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
            print("Merging for model scenario: " + model_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_df = gpd.read_file(file)[ROAD_LINKS_VOL_COLS].rename(columns=rename_columns(model_code, ROAD_LINKS_VOL_COLS))
                        base_df = base_df.merge(temp_df, on='ID', how='left')
                        
        link_vol_dfs[period] = base_df
        del base_df

    return link_vol_dfs

def get_road_volume_time(links_dfs):
    
    for p, period in enumerate(PERIOD_MAP):
        period_code = PERIOD_MAP[period]
        if p == 0:
            base_df = links_dfs[period].copy()
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
            print("Merging for model scenario for PV: " + model_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_df = gpd.read_file(file)[ROAD_LINKS_VOL_COLS].rename(columns=rename_columns(model_code, ROAD_LINKS_VOL_COLS))
                        base_df = base_df.merge(temp_df, on='ID', how='left')
                        

    return base_df

msm_veh_link_vols_df = get_road_volume_time(msm_road_links_dfs)

Merging for model scenario for PV: 18112
Merging for model scenario for PV: 26116
Merging for model scenario for PV: 26176
Merging for model scenario for PV: 26150
Merging for model scenario for PV: 26152
Merging for model scenario for PV: 26154
Merging for model scenario for PV: 26156
Merging for model scenario for PV: 26158
Merging for model scenario for PV: 26164
Merging for model scenario for PV: 26166
Merging for model scenario for PV: 26174
Merging for model scenario for PV: 26170
Merging for model scenario for PV: 26172
Merging for model scenario for PV: 26178
Merging for model scenario for PV: 26180
Merging for model scenario for PV: 26182
Merging for model scenario for PV: 18212
Merging for model scenario for PV: 26216
Merging for model scenario for PV: 26276
Merging for model scenario for PV: 26250
Merging for model scenario for PV: 26252
Merging for model scenario for PV: 26254
Merging for model scenario for PV: 26256
Merging for model scenario for PV: 26258
Merging for mode

In [6]:
def get_pt_volume_time(links_df):
    
    for p, period in enumerate(PERIOD_MAP):
        period_code = PERIOD_MAP[period]
        if p == 0:
            base_df = links_df.copy()
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
            print("Merging for model scenario for PT: " + model_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_tsegs.shp'):
                        temp_df = gpd.read_file(file)
                        temp_df['ID'] = temp_df['INODE'].astype(str) + "-" + temp_df['JNODE'].astype(str)
                        pt_link_temp_df = temp_df.groupby('ID').agg({'VOLTR': 'sum', 'TIMTR': 'first'}).reset_index()
                        pt_link_temp_df = pt_link_temp_df[PT_LINKS_VOL_COLS].rename(columns=rename_columns(model_code, PT_LINKS_VOL_COLS))
                        base_df = base_df.merge(pt_link_temp_df, on='ID', how='left')
                        
    return base_df

msm_link_vols_df = get_pt_volume_time(msm_veh_link_vols_df)

Merging for model scenario for PT: 18112
Merging for model scenario for PT: 26116
Merging for model scenario for PT: 26176
Merging for model scenario for PT: 26150
Merging for model scenario for PT: 26152
Merging for model scenario for PT: 26154
Merging for model scenario for PT: 26156
Merging for model scenario for PT: 26158
Merging for model scenario for PT: 26164
Merging for model scenario for PT: 26166
Merging for model scenario for PT: 26174
Merging for model scenario for PT: 26170
Merging for model scenario for PT: 26172
Merging for model scenario for PT: 26178
Merging for model scenario for PT: 26180
Merging for model scenario for PT: 26182
Merging for model scenario for PT: 18212
Merging for model scenario for PT: 26216
Merging for model scenario for PT: 26276
Merging for model scenario for PT: 26250
Merging for model scenario for PT: 26252
Merging for model scenario for PT: 26254
Merging for model scenario for PT: 26256
Merging for model scenario for PT: 26258
Merging for mode

In [7]:
msm_link_vols_df

,ID,INODE,JNODE,LENGTH,TYPE,MODES,LANES,geometry,NODE_ID,MSM2018,...,VOLTR_26370,TIMTR_26370,VOLTR_26372,TIMTR_26372,VOLTR_26378,TIMTR_26378,VOLTR_26380,TIMTR_26380,VOLTR_26382,TIMTR_26382
0,1-10434,1,10434,8.947524,1,abw,1.0,"LINESTRING (1744722.512 5988249.515, 1735792 5...",10434,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1-10448,1,10448,5.237727,1,abw,1.0,"LINESTRING (1744722.512 5988249.515, 1747206 5...",10448,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-10425,2,10425,0.674503,1,abw,1.0,"LINESTRING (1736211.418 5982441.177, 1736501 5...",10425,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-10427,2,10427,0.857741,1,abw,1.0,"LINESTRING (1736211.418 5982441.177, 1736562 5...",10427,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3-10427,3,10427,2.108739,1,abw,1.0,"LINESTRING (1737358.501 5985176.528, 1736562 5...",10427,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19793,15105-13058,15105,13058,0.087201,7,r,1.0,"LINESTRING (1772305 5890125, 1772357 5890195)",13058,560,...,324.38347,0.000,323.88551,0.000,327.52717,0.000,323.81157,0.000,324.31641,0.000
19794,15105-15104,15105,15104,0.040224,7,r,1.0,"LINESTRING (1772305 5890125, 1772282 5890092)",15104,560,...,2241.44007,0.292,2229.89126,0.292,2281.84423,0.292,2233.68013,0.292,2231.19881,0.292
19795,5242-7334,5242,7334,0.911615,16,a,1.0,"LINESTRING (1758186 5920245, 1757655 5919504)",7334,264,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19796,5340-7334,5340,7334,0.513016,16,a,1.0,"LINESTRING (1757856 5919976, 1757655 5919504)",7334,264,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
metrics_by_tp = []

for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
        for trip_purpose in ['All']:
            temp_connector_df = msm_link_vols_df[(msm_link_vols_df['TYPE']==1)]
            temp_road_df = msm_link_vols_df[(msm_link_vols_df['TYPE']>10)]
            # vkt = (temp_df['LENGTH']*temp_df[f'VOLAU_{model_code}']).sum()
            # vht = (temp_df[f'TIMAU_{model_code}']*temp_df[f'VOLAU_{model_code}']).sum()
            tolled_road_vehicles = ((temp_road_df[f'@toll_{model_code}'] >= 3)*(temp_road_df[f'VOLAU_{model_code}'])).sum()
            half_tolled_road_vehicles = ((temp_road_df[f'@toll_{model_code}'] == 1.55)*(temp_road_df[f'VOLAU_{model_code}'])).sum()
            tolled_connector_vehicles = ((temp_connector_df[f'@toll_{model_code}'] >= 3)*(temp_connector_df[f'VOLAU_{model_code}'])).sum()
            half_tolled_connector_vehicles = ((temp_connector_df[f'@toll_{model_code}'] == 1.55)*(temp_connector_df[f'VOLAU_{model_code}'])).sum()
        metrics_by_tp.append([str(period_code), str(scen_code), REVERSE_PERIOD_MAP[period_code], REVERSE_SCENARIO_MAP[scen_code], trip_purpose, tolled_road_vehicles, half_tolled_road_vehicles, tolled_connector_vehicles, half_tolled_connector_vehicles])

network_metrics_tp_df = pd.DataFrame(metrics_by_tp, columns = ['period_code', 'scen_code', 'Period', 'Scenario', 'Trip_Purpose', 'charged_road_VOLAU_3.1', 'charged_road_VOLAU_1.5', 'charged_connector_VOLAU_3.1', 'charged_connector_VOLAU_1.5'])
network_metrics_tp_df.to_csv(f'{CONGESTION_METRICS_DIR}/tolled_vehs.csv', index=None)

In [10]:
metrics_by_tp = []

for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
        for sector in msm_link_vols_df['Sector_3'].unique().tolist():
            # for trip_purpose in ['hbw', 'hbsh', 'eb', 'hcv', 'All']:
            for trip_purpose in ['All']:
                for road_type in msm_link_vols_df['Group_2'].unique():
                    temp_df = msm_link_vols_df[(msm_link_vols_df['Sector_3']==sector) & (msm_link_vols_df['Group_2']==road_type)]
                    if trip_purpose == 'All':
                        # vkt = (temp_df['LENGTH']*temp_df[f'VOLAU_{model_code}']).sum()
                        # vht = (temp_df[f'TIMAU_{model_code}']*temp_df[f'VOLAU_{model_code}']).sum()
                        tolled_vehicles = ((temp_df[f'@toll_{model_code}'] >= 3)*(temp_df[f'VOLAU_{model_code}'])).sum()
                        half_tolled_vehicles = ((temp_df[f'@toll_{model_code}'] == 1.55)*(temp_df[f'VOLAU_{model_code}'])).sum()
                        tolled_volax = ((temp_df[f'@toll_{model_code}'] >= 3)*(temp_df[f'VOLAX_{model_code}'])).sum()
                        half_tolled_volax = ((temp_df[f'@toll_{model_code}'] == 1.55)*(temp_df[f'VOLAX_{model_code}'])).sum()
                    else:
                        vkt = (temp_df['LENGTH']*temp_df[f'@{trip_purpose}7_{model_code}']).sum()
                        vht = (temp_df[f'TIMAU_{model_code}']*temp_df[f'@{trip_purpose}7_{model_code}']).sum()
                        tolled_vehicles = ((temp_df[f'@toll_{model_code}'] >= 3)*(temp_df[f'@{trip_purpose}7_{model_code}'])).sum() + ((temp_df[f'@toll_{model_code}'] >= 1.55)*(temp_df[f'@{trip_purpose}7_{model_code}'])).sum()/2
                    # metrics_by_tp.append([str(period_code), str(scen_code), REVERSE_PERIOD_MAP[period_code], REVERSE_SCENARIO_MAP[scen_code], trip_purpose, sector, road_type, vkt, vht, tolled_vehicles])
                    

network_metrics_tp_df = pd.DataFrame(metrics_by_tp, columns = ['period_code', 'scen_code', 'Period', 'Scenario', 'Trip_Purpose', 'Sector', 'Road_Type', 'VKT', 'VHT', 'N_Charged_Vehicles'])
network_metrics_tp_df.to_csv(f'{CONGESTION_METRICS_DIR}/network_metrics.csv', index=None)

In [10]:
network_metrics_tp_df['Road_Type'] = network_metrics_tp_df['Road_Type'].str.replace(" ", "").str.replace("/Expressway", "")
network_metrics_tp_df = network_metrics_tp_df[network_metrics_tp_df['Road_Type'].isin(['Motorway', 'Arterial', 'Local', 'Rural'])]
network_metrics_tp_df['model_purpose_metric'] = network_metrics_tp_df['period_code'].astype(str) + \
    network_metrics_tp_df['scen_code'].astype(str) + '_' + \
    network_metrics_tp_df['Trip_Purpose'].astype(str) + '_' + \
    network_metrics_tp_df['Road_Type'].astype(str)
network_metrics_tp_df

,period_code,scen_code,Period,Scenario,Trip_Purpose,Sector,Road_Type,VKT,VHT,N_Charged_Vehicles,model_purpose_metric
3,1,12,AM,2018,hbw,Rodney,Motorway,13816.581588,8859.633408,0.0,112_hbw_Motorway
4,1,12,AM,2018,hbw,Rodney,Arterial,26110.164059,28134.255122,0.0,112_hbw_Arterial
5,1,12,AM,2018,hbw,Rodney,Rural,71487.642850,56049.843470,0.0,112_hbw_Rural
6,1,12,AM,2018,hbw,Rodney,Local,4487.754823,5222.707366,0.0,112_hbw_Local
13,1,12,AM,2018,hbsh,Rodney,Motorway,1109.231154,707.217136,0.0,112_hbsh_Motorway
...,...,...,...,...,...,...,...,...,...,...,...
69586,3,82,PM,Option 3G,hcv,Other Areas,Local,0.000000,0.000000,0.0,382_hcv_Local
69593,3,82,PM,Option 3G,All,Other Areas,Motorway,0.000000,0.000000,0.0,382_All_Motorway
69594,3,82,PM,Option 3G,All,Other Areas,Arterial,0.000000,0.000000,0.0,382_All_Arterial
69595,3,82,PM,Option 3G,All,Other Areas,Rural,0.000000,0.000000,0.0,382_All_Rural


In [12]:
sectors_gdf = MSM_ZONES.dissolve(by='Sector_3').reset_index()[['Sector_3', 'geometry']].rename(columns={'Sector_3': 'Sector'})

In [13]:
vkt_tp = network_metrics_tp_df.pivot(index='Sector', columns='model_purpose_metric', values = 'VKT').add_suffix('_VKT').reset_index()
vht_tp = network_metrics_tp_df.pivot(index='Sector', columns='model_purpose_metric', values = 'VHT').add_suffix('_VHT').reset_index()

In [14]:
sectors_gdf.merge(vkt_tp.merge(vht_tp, on='Sector'), on='Sector').to_crs('epsg:4326').to_file(f'{CONGESTION_METRICS_DIR}/network_metrics_trippurpose_summary.gpkg')

In [49]:
msm_road_vols_df = msm_link_vols_df[msm_link_vols_df['TYPE']>=4]

In [50]:
vcv_cols = [x for x in msm_road_vols_df.columns if '@vcv' in x]
volau_cols = [x for x in msm_road_vols_df.columns if 'VOLAU' in x]
timau_cols = [x for x in msm_road_vols_df.columns if 'TIMAU' in x]

# select_link_analysis = msm_road_vols_df[['ID', 'INODE', 'JNODE', 'LENGTH', 'TYPE', 'LANES', 'Group_2'] + vcv_cols + volau_cols + timau_cols]
# OUTPUT_DIR = '/Users/tszchun.chow/Library/CloudStorage/OneDrive-SharedLibraries-AucklandTransport/Congestion Charging - General/Lead Advisor (shared folder)/02 Analysis (Transport Modelling)/Stage B Options Assessment/Localised Impacts'
# select_link_analysis.to_csv(f'{OUTPUT_DIR}/Link_Volumes+Time.csv')

In [51]:
def congested_link(row, suffix = ""):
    vcv_col = "vcv" + suffix
    if row[vcv_col] >= 0.82 and 11<= row['TYPE'] <= 17:
        return 1
    elif row[vcv_col] >= 0.9 and 18 <= row['TYPE'] <=21 :
        return 1
    elif row[vcv_col] >= 0.58 and 23 <= row['TYPE'] <= 27:
        return 1
    elif row[vcv_col] >= 0.46 and row['TYPE']==22:
        return 1
    else:
        return 0

def export_geoparquet(geo_package, output_path = None, filename = None):
    geo_package.to_crs("EPSG:4326", inplace=True)
    geo_package['wkt'] = geo_package['geometry'].to_wkt()
    
    geo_dataframe = gpd.GeoDataFrame()
    geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])    
    geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")
    geo_dataframe["ID"] = geo_package["ID"]

    geo_parquet = geo_dataframe.merge(geo_package[[i for i in geo_package.columns if i not in ["geometry", "wkt"]]],
                                    left_on="ID", right_on = "ID").drop(columns=['Sector_3'])

    if output_path is not None and filename is not None:
        geo_parquet.fillna(0, inplace=True)
        geo_parquet.to_parquet(f'{CONGESTION_METRICS_DIR}/{filename}', compression = 'zstd')


def calculate_congestion_values(road_links_df, export = True):
    road_links_df['LANE_KM'] = road_links_df['LANES']*road_links_df['LENGTH']
    road_links_df = road_links_df.fillna(0)
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]

        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
            road_links_df = road_links_df.rename(columns = {"@vcv_" +model_code: "vcv_" + model_code})
            road_links_df['CONGESTED_' + model_code] = road_links_df.apply(lambda row: congested_link(row, "_"+ model_code), axis=1)

    if export:
        if not os.path.exists(CONGESTION_METRICS_DIR):
            os.makedirs(CONGESTION_METRICS_DIR)
        road_links_df.to_file(f'{CONGESTION_METRICS_DIR}/msm_link_volumes_tp.gpkg')

        # gpkg = gpd.read_file(f'{CONGESTION_METRICS_DIR}/msm_links_{period}.gpkg', engine='pyogrio')
        export_geoparquet(road_links_df[[x for x in road_links_df.columns if '7p' not in x]], output_path=CONGESTION_METRICS_DIR, filename = f'msm_link_volumes_car+pt.parquet')
    
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)

            road_links_df['CONGESTED_' + model_code] = road_links_df.apply(lambda row: congested_link(row, "_"+ model_code), axis=1)
            road_links_df['CONG_ROAD_KM_' + model_code] = road_links_df['LENGTH']*road_links_df['CONGESTED_' + model_code]
            road_links_df['CONG_LANE_KM_' + model_code] = road_links_df['LANES']*road_links_df['LENGTH']*road_links_df['CONGESTED_' + model_code]

            road_links_df['VKT_' + model_code] = road_links_df['LENGTH'] * road_links_df['VOLAU_' + model_code]
            road_links_df['VHT_' + model_code] = road_links_df['VOLAU_' + model_code] * road_links_df['TIMAU_' + model_code] / 60
            road_links_df['CONG_VKT_' + model_code] = road_links_df['LENGTH'] * road_links_df['VOLAU_' + model_code] *road_links_df['CONGESTED_' + model_code]
            road_links_df['CONG_VHT_' + model_code] = road_links_df['VOLAU_' + model_code] * road_links_df['TIMAU_' + model_code]*road_links_df['CONGESTED_' + model_code] / 60
            
            for tp in ['hbw', 'hbsh', 'eb', 'hcv']:
                road_links_df[f'VKT_{tp}_{model_code}'] = road_links_df['LENGTH'] * road_links_df[f'@{tp}7_{model_code}']
                road_links_df[f'VHT_{tp}_{model_code}'] = road_links_df[f'@{tp}7_{model_code}'] * road_links_df['TIMAU_' + model_code] / 60
                road_links_df[f'CONG_VKT_{tp}_{model_code}'] = road_links_df['LENGTH'] * road_links_df[f'@{tp}7_{model_code}'] *road_links_df['CONGESTED_' + model_code]
                road_links_df[f'CONG_VHT_{tp}_{model_code}'] = road_links_df[f'@{tp}7_{model_code}'] * road_links_df['TIMAU_' + model_code]*road_links_df['CONGESTED_' + model_code] / 60    

    return road_links_df

msm_link_val_df = calculate_congestion_values(msm_road_vols_df)

/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROp

In [52]:
gpd.read_parquet(os.path.join(CONGESTION_METRICS_DIR, 'msm_link_volumes_car_pt.parquet'))

,geometry,ID,INODE,JNODE,LENGTH,TYPE,MODES,LANES,NODE_ID,MSM2018,...,CONGESTED_26356,CONGESTED_26358,CONGESTED_26364,CONGESTED_26366,CONGESTED_26374,CONGESTED_26370,CONGESTED_26372,CONGESTED_26378,CONGESTED_26380,CONGESTED_26382
0,"LINESTRING (174.75046 -36.82373, 174.74283 -36...",1000-5000,1000,5000,1.448476,19,abi,5.0,5000,254,...,1,1,1,1,1,1,1,1,1,1
1,"LINESTRING (174.73799 -36.67744, 174.71888 -36...",1001-1002,1001,1002,2.140000,16,abw,1.0,1002,48,...,0,0,0,0,0,0,0,0,0,0
2,"LINESTRING (174.71888 -36.68907, 174.73799 -36...",1002-1001,1002,1001,2.140000,16,abw,1.0,1001,49,...,0,0,0,0,0,0,0,0,0,0
3,"LINESTRING (174.71888 -36.68907, 174.71704 -36...",1002-1013,1002,1013,0.955616,16,abw,1.0,1013,48,...,0,0,0,0,0,0,0,0,0,0
4,"LINESTRING (174.71888 -36.68907, 174.71976 -36...",1002-1863,1002,1863,1.500000,26,abw,1.0,1863,48,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16243,"LINESTRING (174.93947 -37.11937, 174.94004 -37...",15105-13058,15105,13058,0.087201,7,r,1.0,13058,560,...,0,0,0,0,0,0,0,0,0,0
16244,"LINESTRING (174.93947 -37.11937, 174.93922 -37...",15105-15104,15105,15104,0.040224,7,r,1.0,15104,560,...,0,0,0,0,0,0,0,0,0,0
16245,"LINESTRING (174.7743 -36.85048, 174.7685 -36.8...",5242-7334,5242,7334,0.911615,16,a,1.0,7334,264,...,0,0,0,0,1,0,0,0,0,0
16246,"LINESTRING (174.77066 -36.85296, 174.7685 -36....",5340-7334,5340,7334,0.513016,16,a,1.0,7334,264,...,0,0,0,0,0,0,0,0,0,0


In [30]:
def generate_congestion_metrics(road_links_df, export = True):
    # congestion_metrics_dfs = {}
    agg_metrics = {}
    agg_metrics['LENGTH'] = 'sum'
    agg_metrics['LANE_KM'] = 'sum'

    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]

        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)

            agg_metrics.update({'VKT_' + model_code: 'sum', 'VHT_' + model_code: 'sum',
                                'CONG_ROAD_KM_' + model_code :'sum', 'CONG_LANE_KM_' + model_code :'sum',
                                'CONG_VKT_' + model_code: 'sum', 'CONG_VHT_' + model_code: 'sum'})
            
            for tp in ['hbw', 'hbsh', 'eb', 'hcv']:
                agg_metrics[f'VKT_{tp}_{model_code}'] = 'sum'
                agg_metrics[f'VHT_{tp}_{model_code}'] = 'sum'
                agg_metrics[f'CONG_VKT_{tp}_{model_code}'] = 'sum'
                agg_metrics[f'CONG_VHT_{tp}_{model_code}'] = 'sum'
            
    congestion_metrics = road_links_df.groupby(['Group_2', 'Sector_3']).agg(agg_metrics).reset_index()

    # for scenario in SCENARIO_MAP:
    #     scen_code = SCENARIO_MAP[scenario]
    #     model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
    #     congestion_metrics['CONG_LENGTH_%_' + model_code] = congestion_metrics['CONG_ROAD_KM_' + model_code]/congestion_metrics['LENGTH']
    #     congestion_metrics['CONG_LANE_%_' + model_code] = congestion_metrics['CONG_LANE_KM_' + model_code]/congestion_metrics['LANE_KM']

    #     if scenario not in [x for x in BASE_SCENARIO_MAP]:
    #         for metric in ['VKT', 'VHT', 'CONG_ROAD_KM', 'CONG_LANE_KM', 'CONG_VKT', 'CONG_VHT', 'CONG_LENGTH_%', 'CONG_LANE_%']:
    #             if scenario == 'Option 3C':
    #                 base_scen_code = BASE_SCENARIO_MAP['Do Minimum 3C']
    #             else:
    #                 base_scen_code = BASE_SCENARIO_MAP['Do Minimum']
    #             base_model_code = str(year(scen_code)) + str(period_code) + str(base_scen_code)

    #             congestion_metrics[f'{metric}_ABSDIFF_{model_code}'] = congestion_metrics[f'{metric}_{model_code}'] - congestion_metrics[f'{metric}_{base_model_code}']
    #             congestion_metrics[f'{metric}_PERCDIFF_{model_code}'] = (congestion_metrics[f'{metric}_{model_code}'] / congestion_metrics[f'{metric}_{base_model_code}']) - 1

    congestion_metrics = congestion_metrics.rename(columns={'Group_2': 'Road Type', 'Sector_3': 'Sector_Name'})
    id_vars = ['Road Type', 'Sector_Name']
    congestion_metrics_melt = pd.melt(congestion_metrics, 
                                     id_vars=id_vars,
                                     value_vars = set(congestion_metrics.columns.tolist()) - set(id_vars),
                                     var_name = 'metrics_scenario', value_name = 'value')
        
    congestion_metrics_melt['metrics'] = congestion_metrics_melt['metrics_scenario'].str.rsplit('_', n=1).str[0]
    congestion_metrics_melt['scenario'] = congestion_metrics_melt['metrics_scenario'].str.split("_").str[-1]
    congestion_metrics_melt = congestion_metrics_melt.drop(columns=['metrics_scenario'])

    # congestion_metrics_dfs[period] = congestion_metrics
    if export:
        output_dir = '../outputs/congestion_metrics'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        congestion_metrics.to_csv(f'{output_dir}/congestion_metrics.csv', index=None)
        # congestion_metrics.to_parquet(f'{output_dir}/congestion_metrics.parquet')

    congestion_metrics_melt.to_csv(f'{output_dir}/congestion_metrics_summary.csv', index=None)
    return congestion_metrics

congestion_metrics_df = generate_congestion_metrics(msm_link_val_df)

In [31]:
congestion_metrics_df# [congestion_metrics_df['Sector_Name']==0]

,Road Type,Sector_Name,LENGTH,LANE_KM,VKT_18112,VHT_18112,CONG_ROAD_KM_18112,CONG_LANE_KM_18112,CONG_VKT_18112,CONG_VHT_18112,...,CONG_VKT_hbsh_26382,CONG_VHT_hbsh_26382,VKT_eb_26382,VHT_eb_26382,CONG_VKT_eb_26382,CONG_VHT_eb_26382,VKT_hcv_26382,VHT_hcv_26382,CONG_VKT_hcv_26382,CONG_VHT_hcv_26382
0,Arterial/Expressway,Albany,57.632172,75.459584,68251.936018,2141.055547,4.355553,6.183403,11096.737833,649.395957,...,1066.510615,86.996506,7724.504280,287.433948,1765.078168,136.011121,4275.733093,173.534323,1048.067428,91.203766
1,Arterial/Expressway,CBD,28.803791,54.486497,35023.114940,1680.377177,2.010000,3.550000,5484.066677,481.476815,...,28.306710,5.086017,8083.029327,412.554057,673.886919,58.390214,1561.412753,108.261453,86.690437,25.402647
2,Arterial/Expressway,City Fringe East,31.881783,45.491783,35209.151086,1596.549389,2.270000,3.950000,6988.380805,505.420073,...,179.161422,12.751087,6584.414269,271.678053,1123.986904,76.439744,1499.684706,59.657676,133.342453,13.053388
3,Arterial/Expressway,City Fringe West,32.219518,44.415518,31683.077235,1070.877338,2.250000,2.430000,4800.677147,225.797930,...,130.074820,7.628851,5019.652873,180.638810,511.957655,28.580870,1072.661371,46.499453,129.785246,14.998358
4,Arterial/Expressway,Devonport - Takapuna,52.200471,59.314507,57653.559936,2237.893477,4.610768,4.466768,9591.714341,879.273092,...,716.343959,64.765301,6860.422316,298.135223,1550.619116,145.602777,1732.141657,80.333747,461.855270,40.125572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Rural,Massey,0.220000,0.440000,684.548425,27.423696,0.110000,0.220000,403.525386,21.697847,...,57.430814,1.584835,126.404966,3.802010,126.404966,3.802010,67.281862,2.103658,67.281862,2.103658
89,Rural,NE North Shore,12.086733,12.186733,9267.431994,136.081327,3.334141,3.384141,5073.672972,72.952670,...,387.575744,11.747632,1167.502340,27.313008,741.952630,20.638415,789.410516,20.697422,503.156423,16.497726
90,Rural,Papakura,27.730000,31.170000,26375.172309,577.027076,6.360000,7.530000,10262.216981,293.110800,...,461.134590,13.700671,4637.084376,96.613624,1719.955328,47.818499,1584.169908,32.177870,502.906201,13.838761
91,Rural,Rodney,605.617989,605.617989,248362.186834,3181.408671,19.941491,19.941491,27374.786405,408.654851,...,2645.375564,46.392414,37671.305445,501.016743,4142.092078,69.625240,14845.448283,197.634459,1797.872975,28.452096
